In [1]:
from setup import init_django

In [2]:
init_django()

In [3]:
from market.models import StockQuote

In [4]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [5]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=days_ago)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

113504

In [6]:
total = 0
for obj in qs:
    total += obj.close_price


In [7]:
total

Decimal('4925579524.1519')

In [8]:
total / qs.count()

Decimal('43395.64706223481110797857344')

In [9]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 1, 'avg_price': Decimal('116569.946414834185')}, {'company': 2, 'avg_price': Decimal('0.15588381499242806663')}, {'company': 3, 'avg_price': Decimal('532.1598878166465621')}, {'company': 4, 'avg_price': Decimal('0.29915955024397143059')}]>

In [15]:
count = 5
ticker = "X:BTCUSD"
rolling_qs = list(qs.filter(company__ticker = ticker).order_by('-time')[:count])
rolling_qs.reverse()
rolling_qs

[<StockQuote: StockQuote object (3407989)>,
 <StockQuote: StockQuote object (3407990)>,
 <StockQuote: StockQuote object (3407991)>,
 <StockQuote: StockQuote object (3407992)>,
 <StockQuote: StockQuote object (3407993)>]

In [21]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = Decimal(total / (i + 1))
    print(i+1, obj.id, obj.close_price, avg)

1 3407989 114116.9000 114116.9000
2 3407990 114141.4000 114129.1500
3 3407991 114134.3800 114130.8933333333333333333333
4 3407992 114141.0200 114133.4250
5 3407993 114180.0000 114142.7400


In [22]:
qs = StockQuote.objects.filter(company__ticker = ticker, time__range=(start_date, end_date))

In [28]:
frame_start = -(count - 1)
ma_val = qs.annotate(
    ma=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        partition_by=[],
        frame=RowRange(start=frame_start, end=0),
    )
).order_by('-time')

In [31]:
for i, obj in enumerate(ma_val[:10]):
    print(i, obj.id, obj.close_price, obj.ma, obj.time)


0 3407993 114180.0000 114142.740000000000 2025-08-05 23:59:00+00:00
1 3407992 114141.0200 114135.524000000000 2025-08-05 23:58:00+00:00
2 3407991 114134.3800 114143.044000000000 2025-08-05 23:57:00+00:00
3 3407990 114141.4000 114149.882000000000 2025-08-05 23:56:00+00:00
4 3407989 114116.9000 114146.602000000000 2025-08-05 23:55:00+00:00
5 3407988 114143.9200 114152.342000000000 2025-08-05 23:54:00+00:00
6 3407987 114178.6200 114163.558000000000 2025-08-05 23:53:00+00:00
7 3407986 114168.5700 114153.184000000000 2025-08-05 23:52:00+00:00
8 3407985 114125.0000 114153.336000000000 2025-08-05 23:51:00+00:00
9 3407984 114145.6000 114160.072000000000 2025-08-05 23:50:00+00:00


In [32]:
ma_vals = qs.annotate(
    ma_5=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        partition_by=[],
        frame=RowRange(start=-(5-1), end=0),
    ),
    ma_20=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        partition_by=[],
        frame=RowRange(start=-(20-1), end=0),
    )
).order_by('-time').first()

In [33]:
print(ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20)

3407993 114180.0000 114142.740000000000 114161.387000000000
